Блок задач на тему ООП

На проверку вы должны сдать только файл, содержащий описание класса и одну строку вне класса (в качестве основной программы):
exec(stdin.read())
И в начале файла:
from sys import stdin

Реализуйте класс Matrix. Он должен содержать:

Конструктор от списка списков. Гарантируется, что списки состоят из чисел, не пусты и все имеют одинаковый размер. Конструктор должен копировать содержимое списка списков, т.е. при изменении списков, от которых была сконструирована матрица, содержимое матрицы изменяться не должно.
Метод __str__ переводящий матрицу в строку. При этом элементы внутри одной строки должны быть разделены знаками табуляции, а строки — переносами строк. При этом после каждой строки не должно быть символа табуляции и в конце не должно быть переноса строки.
Метод size без аргументов, возвращающий кортеж вида (число строк, число столбцов)

Добавьте следующие методы:
__add__ принимающий вторую матрицу того же размера и возвращающий сумму матриц
__mul__ принимающий число типа int или float и возвращающий матрицу, умноженную на скаляр
__rmul__ делающий то же самое, что и __mul__. Этот метод будет вызван в том случае, аргумент находится справа. Можно написать __rmul__ = __mul__

Добавьте в метод __add__ проверку на ошибки в размере входных данных чтобы при попытке сложить матрицы разных размеров было выброшено исключение MatrixError таким образом, чтобы matrix1 поле MatrixError стало первым аргументом __add__ (просто self), а matrix2 — вторым (второй операнд для сложения).
Реализуйте метод transpose, транспонирующий матрицу и возвращающую результат (данный метод модифицирует экземпляр класса Matrix)
Реализуйте статический метод transposed, принимающий Matrix и возвращающий транспонированную матрицу.

Измените метод __mul__ таким образом, чтобы матрицы можно было умножать как на скаляры, так и на другие матрицы. В случае, если две матрицы перемножить невозможно, то тогда выбросьте ошибку MatrixError. Первая матрице в ошибке – это self, вторая – это второй операнд умножения.

Добавьте класс SquareMatrix – наследник Matrix с операцией возведения в степень __pow__, принимающей натуральную (с нулём) степень, в которую нужно возвести матрицу. Используйте быстрое возведение в степень

In [ ]:
import copy
from sys import stdin

class MatrixError(Exception):
    def __init__(self, matrix1, matrix2):
        self.matrix1 = matrix1
        self.matrix2 = matrix2


class Matrix:
    def __init__(self, init_list):
        self.matrix_rows = copy.deepcopy(init_list)
    
    def __str__(self):
        output = []
        for i in self.matrix_rows:
            output.append('\t'.join([str(j) for j in i]))
        return '\n'.join(output)
    
    def __add__(self, matrix_object):
        if not len(self.matrix_rows) == len(matrix_object.matrix_rows):
            raise MatrixError(self, matrix_object)
        s_m_rows = matrix_object.matrix_rows
        new_matrix_list = []
        for i in range(len(s_m_rows)):
            line = []
            for j in range(len(s_m_rows[0])):
                line.append(self.matrix_rows[i][j]+s_m_rows[i][j])
            new_matrix_list.append(line)
        new_matrix = Matrix(new_matrix_list)
        return new_matrix

    def __mul__(self, alpha):
        if isinstance(alpha, Matrix):
            E = self.matrix_rows
            R = copy.deepcopy(alpha.matrix_rows)
            if len(R) != len(E[0]):
                raise MatrixError(self, alpha)
            new_matrix_list = []
            result = [[sum(f*g for f,g in zip(E_row, R_col)) for R_col in zip(*R)] for E_row in E]
            if isinstance(alpha, SquareMatrix) and isinstance(self, SquareMatrix):
                new_matrix = SquareMatrix(result)
            else:
                new_matrix = Matrix(result)
            return new_matrix            
        else:
            new_matrix_list = []
            for i in self.matrix_rows:
                line = []
                for j in i:
                    line.append(j * alpha)
                new_matrix_list.append(line)
            new_matrix = Matrix(new_matrix_list)
            return new_matrix      

    __rmul__ = __mul__

    def size(self):
        return len(self.matrix_rows), len(self.matrix_rows[0])
    
    def transpose(self):
        copy_rows = []
        for i in range(len(self.matrix_rows[0])):
            line = []
            for k in self.matrix_rows:
                line.append(k[i])
            copy_rows.append(line)
        self.matrix_rows = copy.deepcopy(copy_rows)
        return self
    
    @staticmethod
    def transposed(matrix_object):
        new_object = copy.deepcopy(matrix_object)
        return new_object.transpose()
    
class SquareMatrix(Matrix):
    def __pow__(self, n_pow):
        if n_pow >= 1000:
            if n_pow%100 == 0:
                ans = (self**(10*n_pow//100))**10
            else:
                ans = ((self**(10*n_pow//100))**10 * self**(n_pow%100))
            return ans
        elif n_pow > 1 and n_pow < 1000:
            return self * self**(n_pow-1)
        elif n_pow == 1:
            return self
        elif n_pow == 0:
            new_matrix_list = []
            for i in range(len(self.matrix_rows)):
                line = []
                for k in range(len(self.matrix_rows)):
                    if k == i:
                        line.append('1')
                    else:
                        line.append('0')
                new_matrix_list.append(line)
            return SquareMatrix(new_matrix_list)